## Chat with linkedin (with evaluator)

In this notebook, I create a chatbot that enables chatting with my linkedin profile.

Additionally, the responses by the chatbot are **evaluated** by an evaluator LLM before they are displayed to the user. 

If the response isn't satisfactory, it is refined before ensuring satisfactoriness before returning to the user.

At a high level, following steps are followed

0. Base application (prompt, data etc.)
1. Create a custom pydantic class to parse evaluator responses.
2. Create evaluator prompt (system, user role messages)
3. Choose evaluator LLM & create it's client
4. Create a function that evaluates the response and returns object of class in 1.
5. Create a function that optimizes (reruns) reponse from base LLM
6. Create final chat function
7. Launch gradio chatInterface

### Base application (prompt, data etc.)

In [1]:
from dotenv import load_dotenv
from openai import OpenAI
from pypdf import PdfReader
import gradio as gr

d:\PERSONAL\Github\miscellaneous-tasks\chat-with-linkedin-profile\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv(override=True)
openai = OpenAI()

reader = PdfReader("linkedin_docs/linkedin_pdf_cg.pdf")
linkedin = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        linkedin += text
        
with open("linkedin_docs/summary.txt", "r", encoding="utf-8") as f:
    summary = f.read()
    
name = "Chaitanya Gokhale"

system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and LinkedIn profile which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer, say so."

system_prompt += f"\n\n## Summary:\n{summary}\n\n## LinkedIn Profile:\n{linkedin}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."

### Create custom pydantic evaluator class

In [6]:
from pydantic import BaseModel

class Evaluator(BaseModel):
    is_correct: bool
    feedback: str

### Create evaluator prompt

In [3]:
evaluator_system_prompt = f"You are an evaluator LLM. Our base task is a chatbot which enables a discussion of the user with the linkedin profile \
    and personality of the {name} who is a person. Your task is to ensure that the response of this chatbot is appropriate using a criteria \
    The criteria is that the response should be in english, professional and high in relevance to the core topic."
    
def get_evaluator_user_prompt(base_llm_response):     
    evaluator_user_prompt = f"Following is the response from our base LLM. {base_llm_response}. \
        As mentioned in the system prompt, you are an evaluator LLM and your job is to evaluate if the response from the other LLM \
        is as expected. The criteria is that the response should be in english, professional and high in relevance to the core topic. \
        Please respond with two values : true or false and a short reasoning for true/false response."
    return evaluator_user_prompt


### Choose evaluator LLM and create its client

In [4]:
import os
gemini = OpenAI(
    api_key=os.getenv("GOOGLE_API_KEY"), 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

### Create evaluate function

A function that takes in the response of base llm and returns the Evaluator object

In [7]:
def evaluate(base_llm_response, evaluator_system_prompt) -> Evaluator:
    evaluator_user_prompt = get_evaluator_user_prompt(base_llm_response)
    evaluator_messages = [{"role": "system", "content": evaluator_system_prompt}, {"role": "user", "content": evaluator_user_prompt}]
    response = gemini.beta.chat.completions.parse(model="gemini-2.5-flash", messages=evaluator_messages, response_format=Evaluator)
    return response.choices[0].message.parsed

### Create a rerun/optimizer

In [12]:
def optimizer(base_llm_response, feedback, user_query):
    optimizer_system_prompt = system_prompt
    optimizer_system_prompt += f"\n\n Your previous response did not satisfy the evaluation criteria i.e. was rejected. \
        Your response was {base_llm_response}.\
        Following was the reason as to why the criteria was not satisfied {feedback}.\
        Your task now is to re-evaluate your response and send a response that makes sure that your original task is catered to \
        as well as the response should eliminate the reason why the evaluation was rejected."
    
    messages = [{"role": "system", "content": optimizer_system_prompt}, {"role": "user", "content": user_query}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    return response.choices[0].message.content

### Create a final chat function

In [13]:
def chat(message, history):
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
    reply = response.choices[0].message.content
    evaluate_reply = evaluate(reply, evaluator_system_prompt=evaluator_system_prompt)
    if evaluate_reply.is_correct:
        print("Reply evaluated : correct")
    else:
        print(f"Reply evaluated : wrong.\n\nFeedback : {evaluate_reply.feedback}")
        reply = optimizer(reply)
    return reply

### Launch gradio

In [14]:
gr.ChatInterface(chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


Reply evaluated : correct
Reply evaluated : correct
Reply evaluated : correct
